In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score ,f1_score,roc_curve
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2
from data_prep import df_prep  
from data_prep import  NLP_Vectorizer
from data_prep import parse_line
from model_src import NLP_model


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package movie_reviews is already

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
nltk.download('wordnet')
from sklearn.svm import SVC
from xgboost import XGBClassifier

import tensorflow as tf
import tensorflow_hub as hub
import pickle
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk
nltk.download('popular')
from sklearn.model_selection import GridSearchCV
nltk.download('wordnet')
from sklearn.svm import SVC 
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to /Users/nathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/nathan/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading p

In [4]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i

0: marketplace
1: customer_id
2: review_id
3: product_id
4: product_parent
5: product_title
6: product_category
7: star_rating
8: helpful_votes
9: total_votes
10: vine
11: verified_purchase
12: review_headline
13: review_body
14: review_date


In [5]:
#Corpus = df_prep(df,np.mean(df['8']/df['9']),.0)
Corpus = pd.read_csv('data/Books_02_Corpus.csv')
Corpus['text_final'] = Corpus['text_final'].fillna(' ')
Corpus  = Corpus.sample(frac=.01, random_state=1)

In [6]:
Corpus

,Unnamed: 0,text,label,text_final,help_score,help_votes,stars
10061,10061,To appreciate Bill Porter's life and success i...,bad,appreciate bill porter life success far good s...,0.307692,13,1
8681,8681,I've read a lot of Graham Masterton in the pas...,good,read lot graham masterton past excite book gre...,0.900000,10,1
44529,44529,Crutcher goes back to 1968 to make his claims ...,bad,crutcher go back make claim pa want sue physic...,0.099237,131,1
21520,21520,"For your convinience, I will now boil down The...",bad,convinience boil chosen one easily readable pa...,0.434783,92,1
42238,42238,There is no SHORTAGE of black men or white men...,bad,shortage black men white men men always shorta...,0.457143,35,1
...,...,...,...,...,...,...,...
29237,29237,I am a student from Saxton Middle school. Alth...,bad,student saxton middle school although seem per...,0.035714,28,1
40431,40431,"Based on the publisher's synopsis, one would e...",good,base publisher synopsis one would expect book ...,0.933333,15,1
59514,59514,This is utter dreck. I remain at a loss as to ...,bad,utter dreck remain loss hackish series continu...,0.303030,33,1
2285,2285,I rarely review books unless I recommend them....,bad,rarely review book unless recommend however bo...,0.208333,96,1


In [7]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus[['text','text_final','help_score','help_votes','stars']],Corpus['label'],test_size=0.3)

In [8]:
TF_IDF_2K =NLP_Vectorizer('TF_IDF',{'max_features':2000,'ngram_range':(2,2)})

In [9]:
CV = CountVectorizer(max_features=1000,ngram_range=(2,2))

In [10]:
orig_Train_Y = Train_Y
Test_Y = TF_IDF_2K.encode_Y(Test_Y)
Train_Y = TF_IDF_2K.encode_Y(Train_Y)

In [11]:
RF = LogisticRegression()

In [12]:
CV.fit(Train_X['text_final'])
Train_X_Vector = CV.transform(Train_X['text_final'])
Test_X_Vector = CV.transform(Test_X['text_final'])

In [13]:
RF.fit(Train_X_Vector,Train_Y)

/Users/nathan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
importances  =pd.DataFrame()
importances['importance'] = RF.coef_[0]
importances['word'] = CV.get_feature_names()
importances['abs'] = abs(importances['importance'])

In [15]:
importances.sort_values(by='importance',ascending=False).head(50).to_csv('data/winners.csv')
importances.sort_values(by='importance',ascending=True).head(50).to_csv('data/losers.csv')